In [4]:
#ref for downloading and importing from internet : https://stackoverflow.com/questions/6861323/download-and-unzip-file-with-python

import urllib.request
import zipfile
import os

cwd = os.getcwd()
    
url = "https://cricsheet.org/downloads/ipl_csv2.zip"
extract_dir = cwd+"/data"

zip_path, _ = urllib.request.urlretrieve(url)
with zipfile.ZipFile(zip_path, "r") as f:
    f.extractall(extract_dir)

### Import basic libraries

In [5]:
import pandas as pd
import numpy as np

### Load the all IPL matches csv file

In [6]:
df = pd.read_csv(cwd+"/data/all_matches.csv",dtype = {'match_id':'int', 'season':'str','innings':'int', 'ball':'float'},parse_dates=True)
df.season = df.season.replace({"2007/08": "2008","2009/10": "2010","2020/21":"2020"})

In [11]:
import re
ipl_players = pd.DataFrame(columns=["season","team","player_name","identifier"])
for mat_id in df.match_id.unique():
    temp_df = pd.read_csv(f"{cwd}/data/{mat_id}_info.csv",sep='delimiter',engine='python')
    season = temp_df.iloc[4][-1].split(",")[-1]
    exploded_player_list = temp_df[temp_df.columns[0]].loc[20:41]
    people_registry = ",".join(temp_df[temp_df.columns[0]].loc[42:].values)
    for player in exploded_player_list:
        #print(player)
        try:
            df_row = []
            df_row.append(season)
            df_row.append(player.split(",")[2]) #team name
            df_row.append(player.split(",")[3]) #player name
            name = player.split(",")[3]
            df_row.append((re.findall(f"(?<={name},)(\w*)",people_registry))[0])
            #print(df_row)
            a_series = pd.Series(df_row, index = ipl_players.columns)
            ipl_players = ipl_players.append(a_series, ignore_index=True)
            #print("Try Succeeded")
        except:
            #print(f"Error for player {name}")
            pass

In [12]:
ipl_players.drop_duplicates(subset=["season","identifier"],inplace=True)

In [13]:
ipl_players[ipl_players["player_name"]=="PV Tambe"]

,season,team,player_name,identifier
8141,2013,Rajasthan Royals,PV Tambe,6aed7e79
8738,2014,Rajasthan Royals,PV Tambe,6aed7e79
10017,2015,Rajasthan Royals,PV Tambe,6aed7e79
11374,2016,Gujarat Lions,PV Tambe,6aed7e79


In [14]:
people_url = "https://cricsheet.org/register/people.csv"
people_df = pd.read_csv(people_url,dtype = {"key_cricinfo":'str'})

In [15]:
merged_df = ipl_players.merge(people_df,how='left',on='identifier')
merged_df.drop(['key_cricbuzz','key_crichq', 'key_bigbash', 'key_cricinfo_2', 'key_cricingif', 'key_cricketarchive','key_cricketarchive_2', 'key_opta', 'key_opta_2', 'key_pulse','key_pulse_2'],axis = 1 , inplace=True)

In [16]:
merged_df

,season,team,player_name,identifier,name,unique_name,key_cricinfo
0,2007/08,Kolkata Knight Riders,SC Ganguly,725529bc,SC Ganguly,SC Ganguly,28779
1,2007/08,Kolkata Knight Riders,BB McCullum,b8a55852,BB McCullum,BB McCullum,37737
2,2007/08,Kolkata Knight Riders,RT Ponting,7d415ea5,RT Ponting,RT Ponting,7133
3,2007/08,Kolkata Knight Riders,DJ Hussey,fd835ab3,DJ Hussey,DJ Hussey,5766
4,2007/08,Kolkata Knight Riders,Mohammad Hafeez,9ab63e7b,Mohammad Hafeez,Mohammad Hafeez,41434
...,...,...,...,...,...,...,...
2493,2022,Rajasthan Royals,KK Nair,944533a5,KK Nair,KK Nair,398439
2494,2022,Rajasthan Royals,OC McCoy,529eb9e0,OC McCoy,OC McCoy,906783
2495,2022,Punjab Kings,NT Ellis,9eb1455b,NT Ellis,NT Ellis,826915
2496,2022,Mumbai Indians,HR Shokeen,0c9652b0,HR Shokeen,HR Shokeen,1175423


In [17]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from tqdm import tqdm

#ref : https://www.digitalocean.com/community/tutorials/how-to-work-with-web-data-using-requests-and-beautiful-soup-with-python-3

player_db = pd.DataFrame(columns=["key_cricinfo","player_name","batting_style","player_type","bowling_style"])
for player_id in tqdm(merged_df.key_cricinfo.unique()):
    #print("Player ID",player_id)
    url = f"https://www.espncricinfo.com/player/firstname-lastname-{player_id}"
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'html.parser')
    res = soup.select(".ds-grid")
    player_d = []
    player_d.append(player_id)
    fields = []
    answers = []
    for r in res[0]:
        fields.append(r.find("p").text)
        answers.append(r.find("span").text)
    page_dict = dict(zip(fields, answers))
    player_d.append(page_dict["Full Name"])
    player_d.append(page_dict["Batting Style"])
    try:
        player_d.append(page_dict["Playing Role"])
        if(page_dict["Playing Role"]!="Wicketkeeper Batter"):
            player_d.append(page_dict["Bowling Style"])
        else:
            player_d.append("NA")
        a_series = pd.Series(player_d,index = player_db.columns)
        player_db = player_db.append(a_series, ignore_index=True)
    except:
        pass

100%|█████████████████████████████████████████| 641/641 [10:13<00:00,  1.05it/s]


In [18]:
final_player_df = pd.DataFrame()
final_player_df = merged_df.merge(player_db,how='left',on='key_cricinfo')
not_available = final_player_df[final_player_df['player_type'].isna()]

In [19]:
missing_db = pd.DataFrame(columns=["key_cricinfo","player_name","batting_style","player_type","bowling_style"])
for player_id in tqdm(not_available.key_cricinfo.unique()):
    #print("Player ID",player_id)
    url = f"https://www.espncricinfo.com/player/firstname-lastname-{player_id}"
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'html.parser')
    res = soup.select(".ds-grid")
    player_d = []
    player_d.append(player_id)
    fields = []
    answers = []
    for r in res[0]:
        fields.append(r.find("p").text)
        answers.append(r.find("span").text)
    page_dict = dict(zip(fields, answers))
    player_d.append(page_dict["Full Name"])
    player_d.append(page_dict["Batting Style"])
    #if playing role is available and they are not bowlers 
    try:
        player_d.append(page_dict["Playing Role"])
        player_d.append("NA")
        a_series = pd.Series(player_d,index = missing_db.columns)
        missing_db = missing_db.append(a_series, ignore_index=True)
        # if playing role is not available ,we set it as NA in except
    except:
        player_d.append("NA")
        # check if player has bowling style
        try:
            player_d.append(page_dict["Bowling Style"])
            a_series = pd.Series(player_d,index = missing_db.columns)
            missing_db = missing_db.append(a_series, ignore_index=True)
        # if player doesn't have bowling style , then set that also as NA
        except:
            player_d.append("NA")
            a_series = pd.Series(player_d,index = missing_db.columns)
            missing_db = missing_db.append(a_series, ignore_index=True)
            pass
        pass

100%|███████████████████████████████████████████| 79/79 [01:07<00:00,  1.17it/s]


In [20]:
final_player_df = final_player_df.merge(missing_db,how='left',on='key_cricinfo')
final_player_df = final_player_df.fillna("")
final_player_df['batting_style_x'] = final_player_df['batting_style_x']+final_player_df['batting_style_y']
final_player_df.drop(["player_name_x","player_name_y"],axis=1,inplace=True)
final_player_df['player_type_x'] = final_player_df['player_type_x']+final_player_df['player_type_y']
final_player_df['bowling_style_x'] = final_player_df['bowling_style_x']+final_player_df['bowling_style_y']
final_player_df.drop("player_name",axis=1,inplace=True)
final_player_df.drop(['player_type_y','batting_style_y','bowling_style_y'],axis=1,inplace=True)
final_player_df.rename({'batting_style_x':'batting_style','player_type_x':'player_type','bowling_style_x':'bowling_style'},axis=1,inplace=True)

In [21]:
final_player_df

,season,team,identifier,name,unique_name,key_cricinfo,batting_style,player_type,bowling_style
0,2007/08,Kolkata Knight Riders,725529bc,SC Ganguly,SC Ganguly,28779,Left hand Bat,Batter,Right arm Medium
1,2007/08,Kolkata Knight Riders,b8a55852,BB McCullum,BB McCullum,37737,Right hand Bat,Wicketkeeper Batter,NA
2,2007/08,Kolkata Knight Riders,7d415ea5,RT Ponting,RT Ponting,7133,Right hand Bat,Top order Batter,Right arm Medium
3,2007/08,Kolkata Knight Riders,fd835ab3,DJ Hussey,DJ Hussey,5766,Right hand Bat,Allrounder,Right arm Offbreak
4,2007/08,Kolkata Knight Riders,9ab63e7b,Mohammad Hafeez,Mohammad Hafeez,41434,Right hand Bat,Allrounder,Right arm Offbreak
...,...,...,...,...,...,...,...,...,...
2493,2022,Rajasthan Royals,944533a5,KK Nair,KK Nair,398439,Right hand Bat,Top order Batter,Right arm Offbreak
2494,2022,Rajasthan Royals,529eb9e0,OC McCoy,OC McCoy,906783,Left hand Bat,Bowler,Left arm Fast medium
2495,2022,Punjab Kings,9eb1455b,NT Ellis,NT Ellis,826915,Right hand Bat,Bowler,Right arm Fast medium
2496,2022,Mumbai Indians,0c9652b0,HR Shokeen,HR Shokeen,1175423,Right hand Bat,Bowler,Right arm Offbreak


In [22]:
final_player_df.to_csv("player_details.csv")